# save the images as video
# Basler ace camera

In [2]:
# import packages and functions
from pypylon import pylon
from imageio import get_writer
import numpy as np
import os
import cv2
from datetime import datetime
import matplotlib.pyplot as plt
import sys
from camcommands import camOpen

connect camera with OpenCV

In [4]:
# list cameras connected with PC

# 1. get transport layer and all attached devices
tl_factory = pylon.TlFactory.GetInstance()
set_devices = tl_factory.EnumerateDevices() #enumerate devices connected

"""
# 1.1 software issue from pylon. this block may be deleted later
# remove duplicated camera list
set_devices = []
for device in devices:
    if device not in set_devices:
        set_devices.append(device)
    else:
        pass
set_devices = tuple(set_devices)
"""
# 2. check the devices information
if len(set_devices) == 0:
    raise pylon.RuntimeException("No camera connected")
else:
    print("total number of devices:", len(set_devices)) # print total number of devices connected
    for dev_info in set_devices: # 
        if dev_info.GetDeviceClass() == 'BaslerGigE': # check if the connected cameras are GigE model. Otherwise software trigger won't work
            print("using %s as IP %s" % (dev_info.GetModelName(), dev_info.GetIpAddress()))
            break
    else:
        raise EnvironmentError("no GigE device found")

# 3. create instant array and attach all devices on the Array
cams = pylon.InstantCameraArray(len(set_devices)) # create instant
for i, cam in enumerate(cams):
    cam.Attach(tl_factory.CreateDevice(set_devices[i]))


total number of devices: 1
using acA1300-30gc as IP 169.254.91.231


In [5]:
# setting camera's parameters
fps = 10 # frame per sec
ExposureTime = 5000 # in µs
Height, width = 962, 1286 # in pixel

for cam in cams:
    cam.Open() # open camera to change the parameter
    print("Setting device ", cam.GetDeviceInfo().GetFriendlyName())
    cam.Height.SetValue(Height)
    cam.Width.SetValue(width)
    cam.ExposureTimeRaw.SetValue(ExposureTime)
    cam.AcquisitionFrameRateAbs.SetValue(fps)
    print(cam.Height.GetValue(), cam.Width.GetValue(), cam.ExposureTimeRaw.GetValue(), cam.AcquisitionFrameRateAbs.GetValue())

    cam.Close() # close camera

Setting device  acA1300_SN01 (21566554)
962 1286 5000 10.0


In [9]:
"""
working with multiple cameras
for the detail commands, see here: https://docs.baslerweb.com/action-commands.html#action-device-key
short paramer description
Action Device Key: 32-bit key to excute a corresponding action on camera
Action Group Key: 32-bit key for a group of devices (cameras) to excute an action
Action Group Mask: 32-bit key for a subgroup of devies

setting up the connection with multiple cameras and checking the parameters
"""

# 1. set action key, group key and group mask. As I'm using one group and all cameras in this group working simaltaneously,
# I did not set multiple action key, group key and group mask
action_key = 0x4711
group_key = 0x112233
group_mask = pylon.AllGroupMask

# 2. Initiate automatic configuration by registering ActionTriggerConfiguration.
for cam in cams:
    cam.RegisterConfiguration(
        pylon.ActionTriggerConfiguration(action_key, group_key, group_mask),
        pylon.RegistrationMode_Append,
        pylon.Cleanup_Delete
        )

# 3. Create a suitable ActionCommand object. For that a GigETransportLayer object is needed.
gige_tl = tl_factory.CreateTl('BaslerGigE')

# 4. Using default value of "255.255.255.255" for fourth parameter 'broadcastAddress'.
act_cmd = gige_tl.ActionCommand(action_key, group_key, group_mask)

In [10]:
# capturing video with multiple cameras at simultaneously
filename = {0:"front", 1:"side"}

for i, cam in enumerate(cams):
    with get_writer("/Users/dkim/Desktop/basler_camera/recording/%s_multi_cams.avi" % (filename[i] ,) ) as writer:
        cam.StopGrabbing()
        cam.StartGrabbingMax(300)
        while cam.IsGrabbing():
            res = cam.RetrieveResult(300)
            writer.append_data(res.Array)
            #print(res.BlockID)
            res.Release()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fa7c1279000] Warning: data is not aligned! This can lead to a speed loss
